## Import and set constants

In [ ]:
import glob
from dataclasses import dataclass
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline

import math

import torch

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

INPUT_PATH = '/kaggle/input/smartphone-decimeter-2023'

WGS84_SEMI_MAJOR_AXIS = 6378137.0
WGS84_SEMI_MINOR_AXIS = 6356752.314245
WGS84_SQUARED_FIRST_ECCENTRICITY  = 6.69437999013e-3
WGS84_SQUARED_SECOND_ECCENTRICITY = 6.73949674226e-3

HAVERSINE_RADIUS = 6_371_000

## Load Device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

## Class to manage converting between lat/lng BLH and GNSS

In [ ]:
@dataclass
class ECEF:
    x: np.array
    y: np.array
    z: np.array

    def to_numpy(self):
        return np.stack([self.x, self.y, self.z], axis=0)

    @staticmethod
    def from_numpy(pos):
        x, y, z = [np.squeeze(w) for w in np.split(pos, 3, axis=-1)]
        return ECEF(x=x, y=y, z=z)

@dataclass
class BLH:
    lat : np.array
    lng : np.array
    hgt : np.array = 0
        

def ECEF_to_BLH(ecef):
    a = WGS84_SEMI_MAJOR_AXIS
    b = WGS84_SEMI_MINOR_AXIS
    e2  = WGS84_SQUARED_FIRST_ECCENTRICITY
    e2_ = WGS84_SQUARED_SECOND_ECCENTRICITY
    x = ecef.x
    y = ecef.y
    z = ecef.z
    r = np.sqrt(x**2 + y**2)
    t = np.arctan2(z * (a/b), r)
    B = np.arctan2(z + (e2_*b)*np.sin(t)**3, r - (e2*a)*np.cos(t)**3)
    L = np.arctan2(y, x)
    n = a / np.sqrt(1 - e2*np.sin(B)**2)
    H = (r / np.cos(B)) - n
    return BLH(lat=B, lng=L, hgt=H)

def haversine_distance(blh_1, blh_2):
    dlat = blh_2.lat - blh_1.lat
    dlng = blh_2.lng - blh_1.lng
    a = np.sin(dlat/2)**2 + np.cos(blh_1.lat) * np.cos(blh_2.lat) * np.sin(dlng/2)**2
    dist = 2 * HAVERSINE_RADIUS * np.arcsin(np.sqrt(a))
    return dist

def pandas_haversine_distance(df1, df2):
    blh1 = BLH(
        lat=np.deg2rad(df1['LatitudeDegrees'].to_numpy()),
        lng=np.deg2rad(df1['LongitudeDegrees'].to_numpy()),
        hgt=0,
    )
    blh2 = BLH(
        lat=np.deg2rad(df2['LatitudeDegrees'].to_numpy()),
        lng=np.deg2rad(df2['LongitudeDegrees'].to_numpy()),
        hgt=0,
    )
    return haversine_distance(blh1, blh2)


In [ ]:
def ecef_to_lat_lng(tripID, gnss_df, UnixTimeMillis):
    ecef_columns = ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']
    columns = ['utcTimeMillis'] + ecef_columns
    ecef_df = (gnss_df.drop_duplicates(subset='utcTimeMillis')[columns]
               .dropna().reset_index(drop=True))
    ecef = ECEF.from_numpy(ecef_df[ecef_columns].to_numpy())
    blh  = ECEF_to_BLH(ecef)

    TIME = ecef_df['utcTimeMillis'].to_numpy()
    lat = InterpolatedUnivariateSpline(TIME, blh.lat, ext=3)(UnixTimeMillis)
    lng = InterpolatedUnivariateSpline(TIME, blh.lng, ext=3)(UnixTimeMillis)
    return pd.DataFrame({
#         'tripId' : tripID,
        'utcTimeMillis'   : UnixTimeMillis,
        'LatitudeDegrees'  : np.degrees(lat),
        'LongitudeDegrees' : np.degrees(lng),
    })

def calc_score(pred_df, gt_df):
    d = pandas_haversine_distance(pred_df, gt_df)
    score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])    
    return score


In [ ]:
def print_comparison(lat, lng, gt_lat, gt_lng):
    for lat_val, lng_val, gt_lat_val, gt_lng_val in zip(lat, lng, gt_lat, gt_lng):
        print(f'Pred: ({lat_val:<12.7f}, {lng_val:<12.7f}) Ground Truth: ({gt_lat_val:<12.7f}, {gt_lng_val:<12.7f})')
        
def print_batch(amnt, lat_arr, lng_arr, gt_lat_arr, gt_lng_arr):
    for batch in range(amnt):
        print(f'Val data {batch}')
        print_comparison(lat_arr[batch], lng_arr[batch], gt_lat_arr[batch], gt_lng_arr[batch])

## Loading Data

In [ ]:
%%capture --no-stdout

pred_dfs  = []
gt_dfs = []



for dirname in sorted(glob.glob(f'/kaggle/input/smartphone-decimeter-2023/sdc2023/train/*/*')):
    drive, phone = dirname.split('/')[-2:]
    tripID  = f'{drive}/{phone}'
    gnss_df = pd.read_csv(f'{dirname}/device_gnss.csv')
    gt_df   = pd.read_csv(f'{dirname}/ground_truth.csv')
    
    info_cols = ['IonosphericDelayMeters', 'TroposphericDelayMeters']
    columns = ['utcTimeMillis'] + info_cols
    info_df = (gnss_df.drop_duplicates(subset='utcTimeMillis')[columns].fillna(0).reset_index(drop=True))
    
    for col in info_cols:
        info_df[col] = info_df[col].fillna((info_df[col].bfill() + info_df[col].ffill()) / 2)
        
    pred_df = ecef_to_lat_lng(tripID, gnss_df, gt_df['UnixTimeMillis'])
    pred_df = pd.merge(pred_df, info_df, on='utcTimeMillis', how='left')
    gt_df   = gt_df[['LatitudeDegrees', 'LongitudeDegrees']]
    print(tripID)
#     print(pred_df.shape)
#     print(gt_df.shape)
    pred_dfs.append(pred_df)
    gt_dfs.append(gt_df)

## Define Baseline Model

In [ ]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
# time sequence encoder
class TransformerEncoder(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.upscale = torch.nn.Linear(config.input_dim, config.d_model) 
        self.pos_encoder = PositionalEncoding(config.d_model, config.dropout, config.max_seq_len)
        self.transformer = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(
                d_model=config.d_model,
                nhead=config.nhead,
                dim_feedforward=config.dim_feedforward,
                dropout=config.dropout,
                activation=config.activation,
                batch_first=True
            ),
            num_layers=config.num_layers
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(config.d_model, config.d_model),
            torch.nn.ReLU(),
            torch.nn.Linear(config.d_model, config.d_model),
            torch.nn.ReLU(),
            torch.nn.Linear(config.d_model, config.output_dim)
        )
        
        
    def forward(self, x):
        x = self.upscale(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

## Declare Model

In [ ]:
class Config:
    def __init__(self, config_dict):
        self.input_dim = config_dict.get('input_dim')
        self.d_model = config_dict.get('d_model')
        self.nhead = config_dict.get('nhead')
        self.dim_feedforward = config_dict.get('dim_feedforward')
        self.dropout = config_dict.get('dropout')
        self.activation = config_dict.get('activation')
        self.num_layers = config_dict.get('num_layers')
        self.output_dim = config_dict.get('output_dim')
        self.input_path = config_dict.get('input_path')
        self.max_seq_len = config_dict.get('max_seq_len')
        self.val_split = config_dict.get('val_split')

config_dict = {
    "input_dim": 4, 
    "d_model": 8,
    "nhead": 4,
    "dim_feedforward": 128,
    "dropout": 0.1,
    "activation": "relu",
    "num_layers": 6,
    "output_dim": 2, 
    "input_path": "INPUT_PATH",
    "max_seq_len": 3500,
    "val_split": 0.05,
}

config = Config(config_dict)

In [ ]:
model = TransformerEncoder(config)
model.to(device)

## Define Dataset Class

In [ ]:

class GNSSDataset(torch.utils.data.Dataset):

    # dataset constructor.
    def __init__(self, pred_dfs, gt_dfs):
        '''
        pred_dfs (list): list of dataframes containing the chosen columns
        gt_df (pd.DataFrame): dataframe containing the ground truth lat, lng
        '''
        self.pred_dfs = pred_dfs
        self.gt_dfs = gt_dfs
        self.sequences = []
        self.labels = []
        
        for pred_df in self.pred_dfs:
            x_np = pred_df[['LatitudeDegrees', 'LongitudeDegrees', 'IonosphericDelayMeters', 'TroposphericDelayMeters']].to_numpy()
            ## pad to max sequence length
            pad = np.zeros((config.max_seq_len - x_np.shape[0], x_np.shape[1]))
            x_np = np.vstack([x_np, pad])
            #x_np = x_np/180
            self.sequences.append(x_np)
            
        for gt_df in self.gt_dfs:
            y_np = gt_df[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()
            ## pad to max sequence length
            pad = np.zeros((config.max_seq_len - y_np.shape[0], y_np.shape[1]))
            y_np = np.vstack([y_np, pad])
            #y_np = y_np/180
            self.labels.append(y_np)

        self.sequences = np.array(self.sequences, dtype=np.float32)
        self.labels = np.array(self.labels,  dtype=np.float32)
        
        self.labels = self.labels - self.sequences[:, :, :2] # just the residuals
        
        print("seq and label shapes")
        print(self.sequences.shape)
        print(self.labels.shape)

    # return an instance from the dataset
    def __getitem__(self, i):
        # return tuple of ith sequence and label
        return self.sequences[i], self.labels[i]

    # return the size of the dataset
    def __len__(self):
        return self.sequences.shape[0]

## Initialize Dataset

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
print(len(pred_dfs)*config.val_split)

In [ ]:
# train test split into training and validation
X_train, X_val, y_train, y_val = train_test_split(pred_dfs, gt_dfs, 
                            test_size=config.val_split, random_state=2)


train_dataset = GNSSDataset(X_train, y_train)
val_dataset = GNSSDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

## Train the Model

In [ ]:
epochs = 1000
n_eval = 10

In [ ]:
# Initalize optimizer (for gradient descent) and loss function
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.00001)
loss_fn = torch.nn.MSELoss()

step = 0

PATH = "model.pt"
best_loss = 99999999 #high number

# Regularization strength
lambda_l1 = 0.1
lambda_l2 = 0.25

for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")

    # Loop over each batch in the dataset
    for batch in tqdm(train_loader):
        
        optimizer.zero_grad()

        # Unpack the data and labels
        features, labels = batch
        
        features = features.to(device)
        labels = labels.to(device)

        # Forward propagate

        outputs = model(features)

        # Backpropagation and gradient descent            

        loss = loss_fn(outputs, labels)
        # L1 regularization
        l1_reg = sum(param.abs().sum() for param in model.parameters())
        # L2 regularization
        l2_reg = sum(param.pow(2).sum() for param in model.parameters())

        # Add the regularization terms to the loss
        loss += lambda_l1 * l1_reg + lambda_l2 * l2_reg

        loss.backward()
        optimizer.step()
        
        print(f'Loss/train {loss}')

        # Periodically evaluate our model + log to Tensorboard
        if step % n_eval == 0:
            # Compute training loss and metrics
            
            model.eval()
            mean_dist = 0
            mean_score = 0
            mean_raw_score = 3.218092441558838 # not passed thru model
            count = 0
            val_loss = 0
            for features, labels in val_loader:
#                 print(features.shape)
#                 print(features[0, :20])
                features = features.to(device)
                labels = labels.to(device)
                
                val_pred = model(features)
                
                val_loss = loss_fn(val_pred, labels)
                
                features = features.detach().cpu()# * 180
                val_pred = val_pred.detach().cpu()# * 180
                labels = labels.detach().cpu()# * 180
                print(val_pred.shape)
                pred_lats = val_pred[:, :, 0] + features[:, :, 0]
                pred_lngs = val_pred[:, :, 1] + features[:, :, 1]
                gt_lats = labels[:, :, 0] + features[:, :, 0]
                gt_lns = labels[:, :, 1] + features[:, :, 1]
                
                start = 60
                cutoff = 65
                # print some samples
                print_batch(1, pred_lats[:, start:cutoff], pred_lngs[:, start:cutoff], gt_lats[:, start:cutoff], gt_lns[:, start:cutoff]) 
                
                blh1 = BLH(np.deg2rad(pred_lats), np.deg2rad(pred_lngs), hgt = 0)
                blh2 = BLH(np.deg2rad(gt_lats), np.deg2rad(gt_lns), hgt = 0)
                d = haversine_distance(blh1, blh2)
                mean_score += np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])    
                mean_dist += d
#                 blh1 = BLH(np.deg2rad(features[:, :, 0]), np.deg2rad(features[:, :, 1]), hgt = 0)
#                 d = haversine_distance(blh1, blh2)
#                 mean_raw_score += np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])    
                
                count += 1
                
                features = features.cpu()
                labels = labels.cpu()
                
            if(val_loss < best_loss):
                best_loss = val_loss
                
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'loss': val_loss,
                            }, PATH)

                
#             print(f'Val mean dist {mean_dist.mean()}')
            print(f'Val mean score {mean_score} vs baseline {mean_raw_score}')
            print(f'Loss/val {val_loss}')

            #turn on training, evaluate turns off training
            model.train()

        step += 1
